**OBSERVACIÓN: Se llevará a cabo un experimento utilizando diversos algoritmos de regresión de Machine Learning utilizando Apache Spark con el objetivo de predecir la variable 'High' para cumplir con las consignas del proyecto. Este experimento solo se tomará en cuenta la empresa Apple y tiene como finalidad demostrar la necesidad de emplear un modelo de Deep Learning basado en redes LSTM para realizar pronósticos sobre los valores máximos de las acciones de las empresas. En este proceso se considerarán las mismas variables de entrada y salida que el modelo de DL**

### **Multiple Linear Regression**

In [1]:
# Importación de librerías
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler, Normalizer
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator

In [2]:
# Creación de una sesión de Apache Spark
spark=SparkSession.builder.getOrCreate()

In [3]:
# Carga de datos
df=spark.read.csv("../DataStorage/datasets_company_etl.csv", header=True, inferSchema=True)
df.show()

+----------+----------+----------+----------+----------+----------+---------+----------+
|      Date|      Open|      High|       Low|     Close| Adj Close|   Volume|id_company|
+----------+----------+----------+----------+----------+----------+---------+----------+
|2013-01-02| 19.779285| 19.821428| 19.343929| 19.608213| 16.791185|560518000|      AAPL|
|2013-01-02|     27.25|     27.73|     27.15| 27.620001| 22.620338| 52899300|      MSFT|
|2013-01-02|  2.333333|  2.363333|     2.314|  2.357333|  2.357333| 17922000|      TSLA|
|2013-01-02|    12.804|    12.905|    12.663|   12.8655|   12.8655| 65420000|      AMZN|
|2013-01-02| 17.918339|  18.10713| 17.846855| 18.013729| 18.013729|102033017|      GOOG|
|2013-01-02|185.554489|187.715103|185.277252|187.715103| 122.39386|  4428869|       IBM|
|2013-01-02| 27.440001|     28.18|     27.42|      28.0|      28.0| 69846400|      META|
|2013-01-03| 27.629999|     27.65|     27.16|     27.25| 22.317314| 48294400|      MSFT|
|2013-01-03| 27.87999

In [4]:
# Tipo de datos
df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- id_company: string (nullable = true)



In [5]:
# Cantidad de filas y columnas
df.count(), len(df.columns)

(27384, 8)

In [6]:
# Función para la selección de variables y para la separacion en entrenamiento y validación
def randomSplit(df, company):
  df2=df.filter(df.id_company==company)
  set_train=df2.filter(col("Date")<="2021-12-31")
  set_test=df2.filter(col("Date")>"2021-12-31")
  set_train=set_train.select(set_train.columns[1:5])
  set_test=set_test.select(set_test.columns[1:5])

  return set_train, set_test

In [7]:
set_train, set_test = randomSplit(df, "AAPL")

In [9]:
# Creación del Pipeline
va=VectorAssembler(inputCols=set_train.columns, outputCol="features")
n=Normalizer(inputCol="features", outputCol="features_nom")
lr=LinearRegression(featuresCol="features_nom", labelCol="High")

p=Pipeline(stages=[va,n,lr])
model=p.fit(set_train)

In [10]:
# Predicciones con el set de entrenamiento
set_pred_train=model.transform(set_train)

In [11]:
# Predicciones con el set de evaluación
set_pred_test=model.transform(set_test)
set_pred_test.show(truncate=False)

+----------+----------+----------+----------+---------------------------------------------+-------------------------------------------------------------------------------+------------------+
|Open      |High      |Low       |Close     |features                                     |features_nom                                                                   |prediction        |
+----------+----------+----------+----------+---------------------------------------------+-------------------------------------------------------------------------------+------------------+
|177.830002|182.880005|177.710007|182.009995|[177.830002,182.880005,177.710007,182.009995]|[0.49363508139088613,0.5076532932443012,0.4933019894439406,0.5052382482444091] |62.18507756150211 |
|177.830002|182.880005|177.710007|182.009995|[177.830002,182.880005,177.710007,182.009995]|[0.49363508139088613,0.5076532932443012,0.4933019894439406,0.5052382482444091] |62.18507756150211 |
|177.830002|182.880005|177.710007|182.009995|

In [12]:
# Comparando los valores
set_pred_test.select("High","prediction").show()

+----------+------------------+
|      High|        prediction|
+----------+------------------+
|182.880005| 62.18507756150211|
|182.880005| 62.18507756150211|
|182.880005| 62.18507756150211|
|182.940002|  52.6776828422735|
|180.169998| 57.88863761222456|
|175.300003|54.972590461256914|
|174.139999| 52.48100206973322|
|     172.5| 60.01063745038118|
|     172.5| 60.01063745038118|
|     172.5| 60.01063745038118|
|175.179993| 59.86932619760046|
|177.179993|47.935672523177345|
|176.619995| 57.22060401630006|
|173.779999| 51.12320909727714|
|172.539993| 51.58429641480325|
|172.539993| 51.58429641480325|
|172.539993| 51.58429641480325|
|172.539993| 51.58429641480325|
|171.080002|58.894150591266225|
|169.679993|62.877467076454195|
+----------+------------------+
only showing top 20 rows



Se observa que los valores predichos de `High` difieren demasiado de los valores reales

In [13]:
# Evaluando el modelo
rmse_train=RegressionEvaluator(labelCol="High", predictionCol="prediction", metricName="rmse").evaluate(set_pred_train)
r2_train=RegressionEvaluator(labelCol="High", predictionCol="prediction", metricName="r2").evaluate(set_pred_train)

rmse_test=RegressionEvaluator(labelCol="High", predictionCol="prediction", metricName="rmse").evaluate(set_pred_test)
r2_test=RegressionEvaluator(labelCol="High", predictionCol="prediction", metricName="r2").evaluate(set_pred_test)

print("Raíz del error cuadrático medio (RMSE) Train/Test: ", rmse_train, rmse_test)
print("Coeficiente de determinación (R^2) Train/Test:",r2_train, r2_test)

Raíz del error cuadrático medio (RMSE) Train/Test:  39.172379547162436 108.42198179714659
Coeficiente de determinación (R^2) Train/Test: 0.03916443405997194 -41.99349921874794


**CONCLUSIÓN:** El modelo de regresión lineal múltiple se desempeña deficientemente, ya que presenta un elevado RMSE en los conjuntos de entrenamiento y prueba, alejándose considerablemente de 0. Asimismo, los valores de R^2 para el entrenamiento y la prueba distan significativamente de 1, incluso alcanzando valores negativos en el conjunto de prueba. Esta situación refleja la presencia tanto de overfitting como de underfitting. El overfitting se evidencia en las notables diferencias entre los valores de RMSE y R^2 en los conjuntos de entrenamiento y validación. Por otro lado, el underfitting se manifiesta en la incapacidad del modelo para acercarse a los valores ideales de un RMSE cercano a 0 y un R^2 cercano a 1.

---

### **Decision Tree Regression**

In [ ]:
# Importación de librerías
from pyspark.ml.regression import DecisionTreeRegressor

In [18]:
# Creación del Pipeline
va=VectorAssembler(inputCols=set_train.columns, outputCol="features")
n=Normalizer(inputCol="features", outputCol="features_nom")
dtc=DecisionTreeRegressor(featuresCol="features_nom", labelCol="High")

p=Pipeline(stages=[va,n,dtc])
model=p.fit(set_train)

In [19]:
# Predicciones con el set de entrenamiento y prueba
set_pred_train=model.transform(set_train)
set_pred_test=model.transform(set_test)

In [20]:
# Comparando los valores
set_pred_test.select("High","prediction").show()

+----------+------------------+
|      High|        prediction|
+----------+------------------+
|182.880005| 60.24960954362414|
|182.880005| 60.24960954362414|
|182.880005| 60.24960954362414|
|182.940002| 39.69195310897436|
|180.169998| 49.36390479999997|
|175.300003|59.788804069767444|
|174.139999| 49.59807939955356|
|     172.5| 84.75508079130434|
|     172.5| 84.75508079130434|
|     172.5| 84.75508079130434|
|175.179993| 55.01543199246704|
|177.179993| 49.59807939955356|
|176.619995| 71.57900331666666|
|173.779999| 49.59807939955356|
|172.539993| 39.69195310897436|
|172.539993| 39.69195310897436|
|172.539993| 39.69195310897436|
|172.539993| 39.69195310897436|
|171.080002| 71.57900331666666|
|169.679993| 71.57900331666666|
+----------+------------------+
only showing top 20 rows



In [21]:
# Evaluando el modelo
rmse_train=RegressionEvaluator(labelCol="High", predictionCol="prediction", metricName="rmse").evaluate(set_pred_train)
r2_train=RegressionEvaluator(labelCol="High", predictionCol="prediction", metricName="r2").evaluate(set_pred_train)

rmse_test=RegressionEvaluator(labelCol="High", predictionCol="prediction", metricName="rmse").evaluate(set_pred_test)
r2_test=RegressionEvaluator(labelCol="High", predictionCol="prediction", metricName="r2").evaluate(set_pred_test)

print("Raíz del error cuadrático medio (RMSE) Train/Test: ", rmse_train, rmse_test)
print("Coeficiente de determinación (R^2) Train/Test:",r2_train, r2_test)

Raíz del error cuadrático medio (RMSE) Train/Test:  37.97788866350019 109.2935491357414
Coeficiente de determinación (R^2) Train/Test: 0.09686890220765776 -42.68749761825537


---

### **Random Forest Regression**

In [22]:
# Importación de librerías
from pyspark.ml.regression import RandomForestRegressor

In [23]:
# Creación del Pipeline
va=VectorAssembler(inputCols=set_train.columns, outputCol="features")
n=Normalizer(inputCol="features", outputCol="features_nom")
rfr=RandomForestRegressor(featuresCol="features_nom", labelCol="High")

p=Pipeline(stages=[va,n,rfr])
model=p.fit(set_train)

In [24]:
# Predicciones con el set de entrenamiento y prueba
set_pred_train=model.transform(set_train)
set_pred_test=model.transform(set_test)

In [25]:
# Comparando los valores
set_pred_test.select("High","prediction").show()

+----------+------------------+
|      High|        prediction|
+----------+------------------+
|182.880005|60.418298149815016|
|182.880005|60.418298149815016|
|182.880005|60.418298149815016|
|182.940002| 43.72108920098153|
|180.169998| 50.08421014621072|
|175.300003|56.306436315952816|
|174.139999| 51.89232093337175|
|     172.5| 73.44624403945694|
|     172.5| 73.44624403945694|
|     172.5| 73.44624403945694|
|175.179993| 74.80906281849482|
|177.179993|51.539075159885286|
|176.619995|62.684684616115874|
|173.779999| 51.62433304738249|
|172.539993|  46.5966134665981|
|172.539993|  46.5966134665981|
|172.539993|  46.5966134665981|
|172.539993|  46.5966134665981|
|171.080002| 63.85941027895376|
|169.679993| 83.88254465208314|
+----------+------------------+
only showing top 20 rows



In [26]:
# Evaluando el modelo
rmse_train=RegressionEvaluator(labelCol="High", predictionCol="prediction", metricName="rmse").evaluate(set_pred_train)
r2_train=RegressionEvaluator(labelCol="High", predictionCol="prediction", metricName="r2").evaluate(set_pred_train)

rmse_test=RegressionEvaluator(labelCol="High", predictionCol="prediction", metricName="rmse").evaluate(set_pred_test)
r2_test=RegressionEvaluator(labelCol="High", predictionCol="prediction", metricName="r2").evaluate(set_pred_test)

print("Raíz del error cuadrático medio (RMSE) Train/Test: ", rmse_train, rmse_test)
print("Coeficiente de determinación (R^2) Train/Test:",r2_train, r2_test)

Raíz del error cuadrático medio (RMSE) Train/Test:  37.45921041326973 108.86007874293838
Coeficiente de determinación (R^2) Train/Test: 0.12136924603844179 -42.34164585834472


---

### **Gradient-Boosted Tree Regression**

In [27]:
# Importación de librerías
from pyspark.ml.regression import GBTRegressor

In [28]:
# Creación del Pipeline
va=VectorAssembler(inputCols=set_train.columns, outputCol="features")
n=Normalizer(inputCol="features", outputCol="features_nom")
gbtr=GBTRegressor(featuresCol="features_nom", labelCol="High")

p=Pipeline(stages=[va,n,gbtr])
model=p.fit(set_train)

In [29]:
# Predicciones con el set de entrenamiento y prueba
set_pred_train=model.transform(set_train)
set_pred_test=model.transform(set_test)

In [30]:
# Comparando los valores
set_pred_test.select("High","prediction").show()

+----------+------------------+
|      High|        prediction|
+----------+------------------+
|182.880005| 67.76876226812809|
|182.880005| 67.76876226812809|
|182.880005| 67.76876226812809|
|182.940002| 32.59428646813161|
|180.169998|31.394472565828003|
|175.300003|60.717439382057385|
|174.139999| 50.69435761125713|
|     172.5| 85.20653898079215|
|     172.5| 85.20653898079215|
|     172.5| 85.20653898079215|
|175.179993| 64.06604229801378|
|177.179993| 54.51126298947539|
|176.619995| 77.53511617419689|
|173.779999|61.358932285387944|
|172.539993| 39.77555331475612|
|172.539993| 39.77555331475612|
|172.539993| 39.77555331475612|
|172.539993| 39.77555331475612|
|171.080002| 81.99892242951034|
|169.679993|  93.0338757060349|
+----------+------------------+
only showing top 20 rows



In [31]:
# Evaluando el modelo
rmse_train=RegressionEvaluator(labelCol="High", predictionCol="prediction", metricName="rmse").evaluate(set_pred_train)
r2_train=RegressionEvaluator(labelCol="High", predictionCol="prediction", metricName="r2").evaluate(set_pred_train)

rmse_test=RegressionEvaluator(labelCol="High", predictionCol="prediction", metricName="rmse").evaluate(set_pred_test)
r2_test=RegressionEvaluator(labelCol="High", predictionCol="prediction", metricName="r2").evaluate(set_pred_test)

print("Raíz del error cuadrático medio (RMSE) Train/Test: ", rmse_train, rmse_test)
print("Coeficiente de determinación (R^2) Train/Test:", r2_train, r2_test)

Raíz del error cuadrático medio (RMSE) Train/Test:  33.81002895384922 109.27122947856347
Coeficiente de determinación (R^2) Train/Test: 0.28421886479144787 -42.669655935698906


---

**CONCLUSIÓN GENERAL: Como se pudo observar en cada uno de los algoritmos de regresión evaluados, ninguno de ellos logró alcanzar un rendimiento cercano al del modelo de Deep Learning basado en redes LSTM para la predicción de series de tiempo. Los valores de error cuadrático medio (RMSE) de estos algoritmos se mantuvieron en el rango de [30; 110], y los coeficientes de determinación (R^2) se encontraron en el rango de [-42; 0.29]. En contraste, el modelo de Deep Learning mostró un RMSE promedio de tan solo 4.27 y un R^2 promedio de 0.93, incluso al considerar predicciones a siete días, evidenciando su superioridad.**

**Es importante destacar que, aunque es posible que se pueda mejorar el rendimiento de los algoritmos de regresión mediante la experimentación con hiperparámetros, queda claro que no alcanzarían ni remotamente el nivel de rendimiento del modelo de Deep Learning seleccionado. Por lo tanto, la elección del modelo de Deep Learning se respalda firmemente en base a los resultados obtenidos en esta comparación.**